In [1]:
conda install pytorch torchvision -c pytorch

Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [47]:
import torch
import torchvision
import torch.utils.data
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [48]:
import torchvision

In [49]:
n_epochs = 5
batch_size=100
learning_rate = 0.75

The epochs I tend to choose from 1-10 and the batch size I chose at 100 because it was giving me the best result. The accuracy varied when going with a high value for the batch size so I decreased it, especially considering the values used in the input/hidden/output layers you will see below. Learning rate appeared to not have an impact that significantly as I tried anywhere from the 0-1 range associated with the learning rate. 


In [39]:
# Transforming the data to torch tensors and normalize it 
transform = transforms.Compose([transforms.ToTensor(),
transforms.Normalize((0.1307), ((0.3081)))])

The values when normalizing are the mean of the dataset so I figured it would be best to use those

In [40]:
# Preparing training set and testing set
trainset = torchvision.datasets.MNIST('mnist', train=True, 
download=True, transform=transform)
testset = torchvision.datasets.MNIST('mnist', train=False,download=True, transform=transform)

In [41]:
# Preparing training loader and testing loader
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=0)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
shuffle=False, num_workers=0) 

In [42]:
# Computing the shape of the training set and testing set
trainset_shape = trainloader.dataset.train_data.shape
testset_shape = testloader.dataset.test_data.shape

# Printing the computed shapes
print(trainset_shape, testset_shape)

torch.Size([60000, 28, 28]) torch.Size([10000, 28, 28])


In [43]:
# Computing the size of the minibatch for training set and testing set
trainset_batchsize = trainloader.batch_size
testset_batchsize = testloader.batch_size

# Printing sizes of the minibatch
print(trainset_batchsize, testset_batchsize)

100 100


In [44]:
class Net(nn.Module):
    def __init__(self):    
        # Parameters of the net
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28 * 28 * 1, 115)
        self.fc2 = nn.Linear(115, 75)
        self.fc3 = nn.Linear(75, 40)

    def forward(self, x):    
        # Forward pass
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

When choosing the hidden layer, I was experimenting on which values would work best. It seemed that having the hidden value about 60% higher than the output layer generated the most accurate result.  

In [45]:
model = Net()   
optimizer = optim.Adam(model.parameters(), lr=3e-4)
criterion = nn.CrossEntropyLoss()

for batch_idx, data_target in enumerate(trainloader):
    data = data_target[0]
    target = data_target[1]
    data = data.view(-1, 28 * 28)
    optimizer.zero_grad()

    # Completing a forward pass
    output = model(data)

    # Computing the loss, gradients and changing the weights
    loss = criterion(output, target)
    loss.backward()
    optimizer.step()

In [46]:
model.eval()

for i, data in enumerate(testloader, 0):
    inputs, labels = data
    
    # Putting each image into a vector
    inputs = inputs.view(-1, 28 * 28)
    
    # Doing the forward pass and getting the predictions
    outputs = model(inputs)
    _, outputs = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (outputs == labels).sum().item()
print('The testing set accuracy of the network is: %d %%' % (100 * correct / total))

The testing set accuracy of the network is: 93 %


93% accuracy for this neural network on the testing set.